In [1]:
import numpy as np
import scipy
from typing import Callable, Union

##### Write a funciton which returns approximations based on Riemann, Trapezoid, and Simpson's rule.

This step is pretty simple, we're 1-1 implementing the formulas for each of these methods but just as code. Only other interesting note here is that since python is zero index we can implement things like the left endpoint Riemann sum as is.

In [10]:
def approximate_func(f, a, b, n):
    """

    :param f:
    :param a:
    :param b:
    :param n:
    :return:
    """

    x_vals = np.linspace(a,b,n)
    step_h = (b-a)/(n-1)
    f = f(x_vals)

    riemann_sum = step_h * sum(f[:n-1])
    trapezoid_sum = (step_h/2)*(f[0]+2*sum(f[1:n-1])+f[n-1])
    simpson_sum = (step_h/3)*(f[0]+2*sum(f[:n-2:2])+4*sum(f[1:n-1:2])+f[n-1])

    return riemann_sum, trapezoid_sum, simpson_sum

result = approximate_func(lambda x: np.sin(x), 0, np.pi, 11)

print(result[0])
print(result[1])
print(result[2])

1.9835235375094546
1.9835235375094546
2.0001095173150043


##### Implement the fourth-order Runge Kutta method.
I had issues with the below function with correctly type-setting. In the end, I decided to just default to any. ArrayNP just give the option for our initial state to be an actual set of initial conditions or a number.

It's also possible to do the below with solve_ivp instead using RK45.

In [10]:
ArrayNP = Union[float, np.ndarray]

def runge_kutta(f: Callable, s_0:ArrayNP, t_eval) -> np.ndarray:
    """
    Solve an ODE or system of ODEs (e.g. spring) using the fourth-order Runge Kutta method.
    :param f: The function that returns change of state dS(t)/dt
    :param s_0: Initial conditions of ODE as a float or numpy array
    :param t_eval: A numpy array containing the set of time points
    :return: 1D or ND array representing approximate solution
    """
    # Find number of steps via
    n_steps = len(t_eval)

    # Numpy function that checks for the dimension of the array
    if np.ndim(s_0) == 0:
        s = np.zeros(n_steps)
    else:
        s = np.zeros((n_steps, len(s_0)))

    s[0] = s_0

    for j in range(n_steps-1):
        t_j = t_eval[j]
        s_j = s[j]

        step_h = t_eval[j+1] - t_j

        k_1 = f(t_j, s_j)
        k_2 = f(t_j + step_h/2, s_j + step_h/2 * k_1)
        k_3 = f(t_j + step_h/2, s_j + step_h/2 * k_2)
        k_4 = f(t_j + step_h, s_j + step_h * k_3)

        s[j+1] = s_j + step_h/6 * (k_1 + 2*k_2 + 2*k_3 + k_4)

    return s

##### Next, we consider the ODE dS(t)/dt = cos(t).
We set our initial state to 0, and evaluate the solution at 100 points between 0 and 10.

In [12]:
sol = runge_kutta(lambda t,s: np.cos(t), 0, np.linspace(0,10,100))

vals = sol[:]
print(f"Our approximate values from the initial state were: {vals}")

Our approximate values from the initial state were: [ 0.          0.10083842  0.20064886  0.29841382  0.39313663  0.48385166
  0.56963413  0.64960954  0.72296259  0.78894549  0.84688559  0.89619223
  0.93636276  0.96698766  0.98775473  0.99845226  0.99897121  0.98930627
  0.96955598  0.93992169  0.90070548  0.85230715  0.79522009  0.73002626
  0.65739027  0.57805261  0.49282206  0.40256751  0.30820903  0.21070856
  0.11106004  0.01027934 -0.09060615 -0.19056797 -0.28858707 -0.38366421
 -0.47483013 -0.56115546 -0.64176016 -0.71582253 -0.78258753 -0.84137455
 -0.89158429 -0.93270489 -0.96431715 -0.98609881 -0.99782781 -0.99938459
 -0.99075328 -0.97202186 -0.94338129 -0.90512355 -0.85763864 -0.80141065
 -0.73701278 -0.66510154 -0.58641    -0.50174039 -0.41195585 -0.31797167
 -0.22074598 -0.12126992 -0.0205576   0.0803643   0.18046694  0.27872983
  0.37415124  0.46575842  0.55261749  0.63384297  0.70860682  0.77614688
  0.8357746   0.88688213  0.92894846  0.96154475  0.98433869  0.99709793

This looks accurate. In the future, it would be good to confirm this using matplotlib and compare using the analytical solution.

##### Implement the explicit Euler method as a function.
This implementation isn't too complex. Like other code for numerical methods, we mainly are just keeping track of (in this case) x_j and y_j.

In [27]:
def explicit_euler(f:Callable, y_0:ArrayNP, step_x:float, start:float, n_steps:int):
    x_vals = np.arange(start, ((n_steps * step_x)+step_x)+start, step_x)  # Compute end instead of asking user
    y = np.zeros(len(x_vals))

    y[0] = y_0

    for j in range(len(x_vals)-1):
        x_j = x_vals[j]
        y_j = y[j]

        y[j+1] = y_j + step_x * f(x_j, y_j)

    return y

##### Test the function implementation.
Now, let's see what our results look like for x=0.5 and x=1.

In [28]:
euler_sol = explicit_euler(lambda x, y: x-y, 1, 0.1, 0, 10)

print(f"Value of y(0.5): {euler_sol[5]}")
print(f"Value of y(1): {euler_sol[10]}")

Value of y(0.5): 0.68098
Value of y(1): 0.6973568802


##### Compare with scipy.integrate.solve_ivp.
This is just a simple comparison to see how close our implementation is to the scipy function.

In [8]:
t_ivp_eval = np.arange(0, 1.1, 0.1)
ivp_sol = scipy.integrate.solve_ivp(lambda x,y: x-y, [0, 1.1], [1], t_eval=t_ivp_eval)

print(f".solve_ivp value of y(0.5): {ivp_sol.y[0][5]}")
print(f".solve_ivp value of y(1): {ivp_sol.y[0][10]}")

0.7126593691868135
0.735960241248649


##### Compare with the analytical solution.
This is just a simple comparison to see how close our implementation and the scipy function are.

In [9]:
y_x = lambda x: x-1+(2/np.exp(x))

print(f"Exact value of y(0.5): {y_x(0.5)}")
print(f"Exact value of y(1): {y_x(1)}")

Exact value of y(0.5): 0.7130613194252668
Exact value of y(1): 0.7357588823428847


Overall, it seems like our implementation was the least accurate. I'm not sure exactly why this is, but it's possible scipy uses some other technique in their computation.